# Change Detection Example: Log Ratio

This example shows how the Capella API can be used to fetch a time series stack of data, read data for a given bounding box directly from TileDB Cloud, and apply a log ratio change detection with an accumulator.

In [ ]:
import json

from capella import lee_filter

from matplotlib import rcParams
from matplotlib import pyplot as plt

import numpy as np
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
from rasterio.warp import transform_bounds
from rasterio import windows
from rasterio.crs import CRS
from skimage import exposure
from scipy.ndimage import morphology

# Allow division by zero
np.seterr(divide='ignore', invalid='ignore')

%matplotlib inline

### Set up project variables

In [ ]:
with open('filter.json') as f:
    filters = json.load(f)
    BBOX = filters['bbox']
    
POINTING = 'right' # or 'left'

# Threshold setting for change detection
THRSET = 2 

# Windows sizes for filtering
MORPHWINSIZE = 3 # window size for Morphological filtering
FILTSIZE = 3 # window size for speckle filter

### Use the API to search for Capella SAR data

In [ ]:
result = ! rio capella --credentials credentials.json --area filter.json --collection capella-aerial --limit 100 query
fc = json.loads(result[0])
features = fc['features']

In [ ]:
# loop over feature metadata and filter on direction
fids = []

features=  sorted(features, key = lambda f: f['properties']['datetime'])

for ft in features:
    fid = f"tiledb://capellaspace/{ft['id']}"
    with rasterio.open(fid) as src:
        tags = src.tags()
        img_desc = json.loads(tags['TIFFTAG_IMAGEDESCRIPTION'])
        pointing = img_desc['collect']['radar']['pointing']

        if pointing == POINTING:
            fids.append(fid)

### Build a change heatmap from the time series

Ingests images as part of a merge function, speckle filters the images, performs log ratio change detection, thresholds and saves detection map into an accumulator, process repeats through all images and builds a heatmap of change

In [ ]:
def logratio(old_data, new_data, old_nodata, new_nodata, index, roff, coff):
    # old data is the intersection in the destination for the merge which is either empty or has filtered values already
    mask = ~new_nodata[0]
    
    # set nodata values to zero so as to detect change in these areas
    new_data[new_nodata] = 0
 
    new_data[0] = lee_filter(new_data[0], FILTSIZE)
    
    # check whether old_data has any data
    if np.any(old_data[0]):
        dIx = np.log(old_data[0] / new_data[0]).astype(old_data.dtype)
        # Statistics and thresholding
        # Thresholding is empirically derived, requires manual adjustment of THRSET constant
        thr = np.nanmean(dIx) + THRSET*np.nanstd(dIx)
        dIx[dIx < thr] = 0.0
        dIx[dIx > thr] = 1.0

        # Morphological opening to reduce false alarms    
        w = (MORPHWINSIZE, MORPHWINSIZE)
        dIx = morphology.grey_opening(dIx, size=w)

        # update change detection band based on changes in the new data mask
        old_data[1, mask] += dIx[mask]

    # update destination values with filtered data
    old_data[0, mask] = new_data[0, mask]
  
if (len(fids) > 0):
    datasets = []
    try:
        for fid in fids:
            datasets.append(rasterio.open(fid))

        result, _ = merge(datasets, transform_bounds(CRS.from_epsg(4326), datasets[0].crs, *BBOX),
                          nodata=0, output_count=2, method=logratio)
        cd = result[1, :, :]
    finally:
        for ds in datasets:
            ds.close()
else:
    print('No datasets to merge')

### Display the change detection result

In [ ]:
# generate context image from mosaic
result = ! rio capella --credentials credentials.json --area filter.json --collection rotterdam-aerial-mosaic --limit 1 query
fc = json.loads(result[0])
ft = fc['features'][0]

with rasterio.Env():
    fid = f"tiledb://capellaspace/{ft['id']}"
    with rasterio.open(fid) as src:
        native_bounds = transform_bounds(CRS.from_epsg(4326), src.crs, *BBOX)        
        bounds_window = src.window(*native_bounds)
        bounds_window = bounds_window.intersection(windows.Window(0, 0, src.width, src.height))
        ci = lee_filter(src.read(1, window=bounds_window), FILTSIZE)
        ci = exposure.adjust_log(ci, gain=10)

rcParams['figure.figsize'] = 10,10
fig, ax = plt.subplots(1, 2)
ax[0].imshow(ci, cmap='gray');
ax[0].set_title("Context Image");
ax[1].imshow(cd, cmap='jet');
ax[1].set_title("Change Detection Heatmap");
